<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md)

# Lab 4.2 : Spark SQL



### Overview
Using SQL statements with Spark SQL.   

### Depends On 
None

### Run time
20-30 mins


In [1]:
print('Spark UI running on http://18.208.221.237:' + sc.uiWebUrl.split(':')[2])

Spark UI running on http://18.208.221.237:4040


## STEP 1: Load Clickstream data

In [2]:
clickstreamDF = spark.read.json("/data/click-stream/clickstream.json")
print(clickstreamDF)
clickstreamDF.show()

DataFrame[action: string, campaign: string, cost: bigint, domain: string, ip: string, session: string, timestamp: bigint, user: string]
+-------+-----------+----+-----------------+----+----------+-------------+------+
| action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
+-------+-----------+----+-----------------+----+----------+-------------+------+
|clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
|blocked|campaign_12|   5|     facebook.com|ip_3|session_96|1420070400864|user_5|
|clicked| campaign_3|  54|sf.craigslist.org|ip_9|session_61|1420070401728|user_8|
|blocked|campaign_18| 110|    wikipedia.org|ip_5|session_55|1420070402592|user_6|
|clicked| campaign_6|  15|comedycentral.com|ip_9|session_49|1420070403456|user_4|
|blocked| campaign_9| 139|          cnn.com|ip_8|session_13|1420070404320|user_7|
|blocked| campaign_4| 171|   funnyordie.com|ip_1|session_92|1420070405184|user_9|
|blocked|campaign_17|  20|       amazon.com|

**==> Register dataframe as a table**

In [3]:
clickstreamDF.createOrReplaceTempView("clickstream")
print("created clickstream temp table")

created clickstream temp table


## STEP 2: Querying using SQL


**==> Select all logs**

In [5]:
logs = spark.sql("select * from clickstream")
#'logs' is a dataframe

# display the logs, hint : show
logs.show()

+-------+-----------+----+-----------------+----+----------+-------------+------+
| action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
+-------+-----------+----+-----------------+----+----------+-------------+------+
|clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
|blocked|campaign_12|   5|     facebook.com|ip_3|session_96|1420070400864|user_5|
|clicked| campaign_3|  54|sf.craigslist.org|ip_9|session_61|1420070401728|user_8|
|blocked|campaign_18| 110|    wikipedia.org|ip_5|session_55|1420070402592|user_6|
|clicked| campaign_6|  15|comedycentral.com|ip_9|session_49|1420070403456|user_4|
|blocked| campaign_9| 139|          cnn.com|ip_8|session_13|1420070404320|user_7|
|blocked| campaign_4| 171|   funnyordie.com|ip_1|session_92|1420070405184|user_9|
|blocked|campaign_17|  20|       amazon.com|ip_4|session_13|1420070406048|user_1|
|blocked|campaign_20|  78|        zynga.com|ip_5|session_36|1420070406912|user_3|
|blocked|campaig

**==> Find records with only clicks**


In [6]:
# Find Records with only clicks
# TODO: fix the  table name?
spark.sql("select * from clickstream where action = 'clicked'").show()


+-------+-----------+----+-----------------+----+----------+-------------+------+
| action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
+-------+-----------+----+-----------------+----+----------+-------------+------+
|clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
|clicked| campaign_3|  54|sf.craigslist.org|ip_9|session_61|1420070401728|user_8|
|clicked| campaign_6|  15|comedycentral.com|ip_9|session_49|1420070403456|user_4|
|clicked|campaign_16| 141|    wikipedia.org|ip_1|session_57|1420070409504|user_1|
|clicked| campaign_5|  74|       sfgate.com|ip_6|session_27|1420070412096|user_3|
+-------+-----------+----+-----------------+----+----------+-------------+------+



**==> Count records from each domain, sort the output by most to least**

Sample output:

```
+-----------------+-----+
|           domain|total|
+-----------------+-----+
|      nytimes.com|    1|
|     facebook.com|    1|
|       google.com|    2|
|       amazon.com|    2|
|    wikipedia.org|    3|
+-----------------+-----+
```

In [12]:
# TODO: Try this query here:

spark.sql("select domain, COUNT(*) as total from clickstream group by domain  order by total ").show()

+-----------------+-----+
|           domain|total|
+-----------------+-----+
|      nytimes.com|    1|
|        zynga.com|    1|
|          cnn.com|    1|
|        yahoo.com|    1|
|   funnyordie.com|    1|
|       sfgate.com|    1|
|     facebook.com|    1|
|       amazon.com|    2|
|      youtube.com|    2|
|comedycentral.com|    2|
|       google.com|    2|
|sf.craigslist.org|    2|
|    wikipedia.org|    3|
+-----------------+-----+




## STEP 3: Joining Datasets

**==> Load `domains` dataset and register it to table `domains`**

In [13]:
domainsDF = spark.read.json("/data/click-stream/domain-info.json")
domainsDF.show()
domainsDF.createOrReplaceTempView("domains")

+-----------+-----------------+
|   category|           domain|
+-----------+-----------------+
|   SHOPPING|       amazon.com|
|       NEWS|        bbc.co.uk|
|     SOCIAL|     facebook.com|
|      PHOTO|       flickr.com|
|       NEWS|      foxnews.com|
|     SEARCH|       google.com|
|       NEWS|          npr.org|
|       NEWS|      nytimes.com|
|CLASSIFIEDS|sf.craigslist.org|
|       NEWS|       sfgate.com|
|     SOCIAL|      twitter.com|
|       NEWS|     usatoday.com|
|  REFERENCE|    wikipedia.org|
|     SEARCH|        yahoo.com|
|      VIDEO|      youtube.com|
+-----------+-----------------+



**==> Join `clickstreams` and `domains`**    
Hint : Join query syntax for joining two tables A, B on A.x and A.y is
```
spark.sql("select A.*, B.* from A  join B  ON (A.x = B.y)") 
```

In [14]:
# Write sql query for joining clickstreams and domains
# TODO : fill in the common attribute for  clickstream & domain table
# Hint : domain

s = """
SELECT clickstream.*, domains.*  
from clickstream join domains 
ON (clickstream.???= domains.???)
"""

spark.sql(s).show()

ParseException: "\nmismatched input 'from' expecting <EOF>(line 3, pos 0)\n\n== SQL ==\n\nSELECT clickstream.*, domains.*  \nfrom clickstream join domains \n^^^\nON (clickstream.???= domains.???)\n"


**==> Count traffic per domain category (news, social ..etc)**    
Hint : query the joined datasets


In [ ]:
# TODO: Count traffic per domained category

s = """
SELECT  domains.category, ???  
from clickstream join domains 
ON (clickstream.???= domains.???)
group by ???
order by ??? 
"""

spark.sql(s).show()



## STEP 4: Explore UI
(Your DAG visualization may be slightly different from what we show here)

<img src="../assets/images/5.2c.png" style="border: 5px solid grey; max-width:100%;"/>

<img src="../assets/images/5.2d.png" style="border: 5px solid grey; max-width:100%;"/>

<img src="../assets/images/5.2e.png" style="border: 5px solid grey; max-width:100%;"/>

## Step 5 : Understand Query Execution
We will use **explain** 

In [ ]:
spark.sql("select * from clickstream where cost > 100").explain(extended=True)

In [ ]:
spark.sql("join query here").explain(extended=True)